The purpose of this code is to demonstrate that atrous convolution is not compute-heavy but memory-access limited on GPUs, explaining why such operations are challenging to optimize in CUDA.

In [33]:
%%writefile dilated_convolution.cu
#include <cuda_runtime.h>
#include <cstdio>
#include <cstdlib>

#define CHECK_CUDA(call) \
    if ((call) != cudaSuccess) { \
        fprintf(stderr, "CUDA error at %s:%d\n", __FILE__, __LINE__); \
        exit(1); \
    }


__global__ void regular_access(const float* input, float* output, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // global thread id
    if (idx < N) {
        output[idx] = input[idx] * 2.0f;
    }
}


__global__ void dilated_access(const float* input, float* output, int N, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int dilated_idx = idx * stride;

    if (dilated_idx < N) {
        output[dilated_idx] = input[dilated_idx] * 2.0f;
    }
}

int main() {
    const int N = 1 << 24;           // ~16 million elements
    const int stride = 4;            // dilation factor
    const int blockSize = 256;
    const int gridSize = (N + blockSize - 1) / blockSize; //how many blocks we need

    size_t bytes = N * sizeof(float); //Total memory size in bytes

    // Host memory
    float* h_input = (float*)malloc(bytes);
    float* h_output = (float*)malloc(bytes);

    for (int i = 0; i < N; i++) {
        h_input[i] = 1.0f;
    }

    // Device memory
    float *d_input, *d_output;
    // Allocate memory on the GPU
    // Same size as host arrays
    CHECK_CUDA(cudaMalloc(&d_input, bytes));
    CHECK_CUDA(cudaMalloc(&d_output, bytes));

    //copy input data from CPU to GPU
    CHECK_CUDA(cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));

    float ms;

    CHECK_CUDA(cudaMemset(d_output, 0, bytes));
    cudaEventRecord(start);

    regular_access<<<gridSize, blockSize>>>(d_input, d_output, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);

    printf("Regular access time:  %.3f ms\n", ms);

    CHECK_CUDA(cudaMemset(d_output, 0, bytes));
    cudaEventRecord(start);

    dilated_access<<<gridSize, blockSize>>>(d_input, d_output, N, stride);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);

    printf("Dilated access time (stride=%d): %.3f ms\n", stride, ms);

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);

    return 0;
}


Writing dilated_convolution.cu


In [34]:
!nvcc -arch=native dilated_convolution.cu -o dilated_convolution

In [35]:
!./dilated_convolution

Regular access time:  0.606 ms
Dilated access time (stride=4): 0.938 ms
